In [ ]:
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten
from tensorflow.keras.layers import Input, Multiply, GlobalAveragePooling2D, Reshape, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [ ]:
# 경로 설정
train_metadata_path = os.path.expanduser('/2501ml_data/label/train_label.txt')
test_metadata_path = os.path.expanduser('/2501ml_data/label/test_label.txt')
train_data_path = os.path.expanduser('/2501ml_data/train')
test_data_path = os.path.expanduser('/2501ml_data/test')

In [ ]:
# Mel-spectrogram 추출 함수
def extract_mel_spectrogram(file_path, n_mels=128):
    y, sr = librosa.load(file_path, sr=16000)
    mel_spec = librosa.feature.melspectrogram(y=y[:48000], sr=sr, n_mels=n_mels)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return log_mel_spec  # shape: (128, 94)

# 데이터셋 로딩 함수
def load_dataset(metadata_path, data_path):
    x_data, y_data = [], []
    with open(metadata_path, 'r') as f:
        for line in f:
            spk, file_name, _, _, label = line.strip().split(' ')
            wav_path = os.path.join(data_path, file_name)
            features = extract_mel_spectrogram(wav_path)
            x_data.append(features)
            y_data.append(label)
    return np.array(x_data), np.array(y_data)


# Mel-spectrogram 데이터셋 생성
train_x, train_y = load_dataset(train_metadata_path, train_data_path)
test_x, test_y = load_dataset(test_metadata_path, test_data_path)

# 채널 차원 추가 → (128, 400, 1)
train_x = train_x[..., np.newaxis]
test_x = test_x[..., np.newaxis]

# 라벨 인코딩 + 원-핫 인코딩
le = LabelEncoder()
y_train = le.fit_transform(train_y)
y_test = le.transform(test_y)

y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

# ✅ 확인
print("train_x shape:", train_x.shape)
print("test_x shape:", test_x.shape)
print("y_train_oh shape:", y_train_oh.shape)

In [ ]:
act='leaky_relu'

def se_block(x, reduction=8):
    channels = x.shape[-1]
    se = GlobalAveragePooling2D()(x)
    se = Dense(channels // reduction, activation='relu')(se)
    se = Dense(channels, activation='sigmoid')(se)
    se = Reshape((1, 1, channels))(se)
    return Multiply()([x, se])

def build_attention_cnn(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, kernel_size=(3,3), activation=act, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Conv Block 2
    x = Conv2D(64, kernel_size=(3,3), activation=act, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Conv Block 3 (with attention)
    x = Conv2D(128, kernel_size=(3,3), activation=act, padding='same')(x)
    x = BatchNormalization()(x)
    x = se_block(x)  # attention 추가
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Conv Block 4 (with attention)
    x = Conv2D(256, kernel_size=(3,3), activation=act, padding='same')(x)
    x = BatchNormalization()(x)
    x = se_block(x)  # attention 추가
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation=act)(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

# 입력 형태 및 클래스 수 설정
input_shape = train_x.shape[1:]  # (128, 400, 1)
num_classes = y_train_oh.shape[1]

# 2️ 훈련/검증 분리
train_x_split, val_x, y_train_split, val_y = train_test_split(
    train_x, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# 3️ 모델 생성 및 컴파일
model = build_attention_cnn(input_shape)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4️ 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 5️ 모델 학습
history = model.fit(
    train_x_split, y_train_split,
    epochs=50,
    batch_size=32,
    validation_data=(val_x, val_y),
    callbacks=[early_stop],
    verbose=1
)

# 6️ 최종 평가 (test set)
loss, acc = model.evaluate(test_x, y_test, verbose=0)
print(f"\n 최종 Test Accuracy: {acc:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# 예측
y_pred = model.predict(test_x)
y_pred_classes = np.around(y_pred)
# 성능 지표 출력
print("📊 Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))

# Accuracy
acc = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {acc:.4f}")

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()